<a href="https://colab.research.google.com/github/seyeonjungGit/NLP_Algorithm/blob/main/TfidfVectorizer_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  # 텍스트 전처리
  def preprocessing(self, sequences):
    result = []
    '''
    문제 1-1.
    조건 1 : 소문자로의 변환과 특수문자 제거를 수행
    조건 2 : 토큰화는 white space 단위로 수행
    '''
    from tensorflow.keras.preprocessing.text import text_to_word_sequence
    for s in sequences :  # 한 문장씩 꺼내기
      result.append(list(text_to_word_sequence(s)))  # 소문자로 변환 + 특수문자 제거(아포스트로피는 보존) + 토큰화

    return result
  

  # 어휘 사전 구축
  def fit(self, sequences):
    self.fit_checker = False
    '''
    문제 1-2.
    조건 1: 위에서 만든 preprocessing 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
    조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(self.word_dict)을 생성합니다.
    주어진 코드에 있는 self.word_dict를 활용합니다.
    '''
    tokens = self.preprocessing(sequences)  # 중첩리스트가 반환

    from collections import Counter
    all_words_list = sum(tokens, [])  # 토큰들을 한 리스트에 모두 담기
    vocab = Counter(all_words_list)  # 리스트에서 토큰들의 갯수 세아리기
    vocab_size = len(vocab)    # 토큰 중에 어휘사전구축에 활용할 갯수 설정(여기서는 전체 토큰 활용)
    vocab = vocab.most_common(vocab_size)

    i = 0
    for (word, frequency) in vocab :  # 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여
      i += 1
      self.word_dict[word] = i     # word_dict에 어휘 추가.
    self.fit_checker = True


  # 어휘 사전 활용 -> 정수 인덱싱
  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)  # 중첩리스트가 반환
    if self.fit_checker:
      '''
      문제 1-3.
      조건 1: 어휘 사전(self.word_dict)에 없는 단어는 'oov'의 index로 변환합니다.
      '''
      for i in range(len(tokens)) :  # 한 문장을 불러옴
        pre = []
        for j in tokens[i] :  # 한 단어를 꺼낸다.
          if j in self.word_dict.keys():  
            pre.append(self.word_dict[j]) 
          else:    # 어휘 사전(self.word_dict)에 없는 단어는 'oov'의 index로 변환
            pre.append(self.word_dict['oov'])
        result.append(pre)
      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")


  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [ ]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)  # 이중리스트
    '''
    문제 2-1.
    조건 1: IDF 행렬은 list 형태입니다.
    조건 2: IDF 값은 아래 식을 이용해 구합니다.
    - IDF = (특정단어 t가 등장한 문서의 수) 에 반비례하는 수
    조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    '''

    from math import log

    # 정수인덱싱된 문장을 어휘사전으로 만들기
    vocab = list(set(sum(tokenized, [])))  # [1,2,3,4,...]
    vocab.sort()  # 정렬

    # 총 문서의 수
    N = len(tokenized)

    # 특정 단어 t가 등장한 문서의 수(IDF)
    self.result_idf = []
    for t in vocab :      # 어휘사전에서 한단어씩 꺼내기 e.g) 1 or 2 or 3....
      df = 0
      for one in tokenized:  # 전체에서 한 문장씩 꺼낸다. e.g) [1,2,3,4]
        df += (t in one)   # 한 문장에 해당 단어가 들어가 있다. -> True | False   # 전체문서에서 특정단어 t가 얼만큼 등장?
      
      self.result_idf.append(log(N/(1+df)))  # 해당 단어(t)에 대한 idf 값을 리스트에 추가

    self.fit_checker = True
    

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      '''
      문제 2-2.
      조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
      - tf(d, t) : 문장 d에 단어 t가 나타난 횟수
      조건2 : 문제 2-1( fit())에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
      - tf-idf(d,t) = tf(d,t) * idf(d,t)
      '''
      # 정수인덱싱된 문장을 어휘사전으로 만들기
      vocab = list(set(sum(tokenized, [])))
      vocab.sort()  # 정렬    

      # 총 문서의 수
      N = len(tokenized)
      
      # 한 문서안에 특정단어 t가 몇번 들어가는지 (TF)
      self.result_tf = []
      for d in tokenized:  # 문장 한개를 꺼낸다.
        self.result_tf.append([])
        for t in vocab:  # 어휘사전에서 단어 한개 꺼낸다.
          self.result_tf[-1].append(d.count(t))  # 중첩리스트

      # tfidf 구하기
      self.tfidf_matrix = []
      for tf in self.result_tf :  # TF행렬 하나씩 꺼내기 (문서 1개씩 꺼내기)
        matrix = []
        for i in range(len(vocab)):
          matrix.append(tf[i]*self.result_idf[i])
        self.tfidf_matrix.append(matrix)
      return self.tfidf_matrix

    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)  # 중첩리스트